<a href="https://colab.research.google.com/github/vimuth97/FYP-Brain-Tumor-Classification/blob/main/Basic%20pathology%20model/split_regions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import time
import argparse
import pdb
import pandas as pd
import math
import xml.etree.ElementTree as ET
from xml.dom import minidom
import multiprocessing as mp
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import h5py
import itertools
import pickle
!apt update && apt install -y openslide-tools
!pip install openslide-python 
import openslide 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
def splitRegions(source,regions_dir):
  #image = "CPM19_CBICA_AAB_1.tiff"
  slides = sorted(os.listdir(source))
  slides = [slide for slide in slides if os.path.isfile(os.path.join(source, slide))]
  i=0
  for file in slides:
    if i==1:
      print("Line breaked")
      break
    file_name = file.split(".")
    
    full_path = os.path.join(source, file)
    wsi = openslide.OpenSlide(full_path)

    # Read the image by 4000 x 4000 slides. To identify all the regions following method used.
    wsi_h, wsi_w = wsi.level_dimensions[0]

    width_coor=[i for i in range(0,wsi_w,4000)]
    height_coor=[i for i in range(0,wsi_h,4000)]
    
    # Adding last pixel coordinate to calclute acurate region sizes
    width_coor.append(wsi_w)
    height_coor.append(wsi_h)

    for ht in range (len(height_coor)-1):
      for wd in range (len(width_coor)-1):
        h0,w0 = height_coor[ht], width_coor[wd] 
        newFileName = file_name[0]+"-"+str(h0)+"_"+str(w0)+".tiff"
        savePath = os.path.join(regions_dir, newFileName)
        if not (os.path.isfile(savePath)):
          h_size,w_size = (height_coor[ht+1] - height_coor[ht]) , (width_coor[wd+1] - width_coor[wd])
          wsi_region = wsi.read_region((h0,w0),0,(w_size,h_size))    
          wsi_region.convert('RGB').save(savePath)
          print(newFileName+" file written.")
    i+=1
  print(slides)


splitRegions('gdrive/My Drive/Data_Directory','gdrive/My Drive/Regions')